In [ ]:
import pandas as pd
import os
import  multiprocessing as mp
import subprocess
# d = pd.read_csv("../tmp/test2all.dat",error_bad_lines =False,names=['time','src','dst','proto', 'vol','src_port','dst_port'])
DATAOUTDIR="ntpdata1_1"
DATADIR= "../passive-2016/equinix-chicago/20160121-130000.UTC"  #"../tmp/data"

global DbgV

# tshark -r test.pcap.gz -Y udp -Tfields -e "ip.src" -e "ip.dst" -e "udp.srcport"  -e "udp.dstport" -Eseparator=,
# tshark -r test.pcap.gz -Y tcp -Tfields -e "ip.src" -e "ip.dst" -Taggregator=,

def worker(f,ii):
    # global DbgV
    # d = pd.read_csv(DATADIR+"/"+f, error_bad_lines =False,names=['time','src','dst','proto', 'vol','src_port','dst_port'])
    # sld=d[(d['proto']=='UDP')]
    # DbgV=d
    # sld['vol']=sld['vol'].astype(int)
    # grouped = sld['vol'].groupby(sld['src_port'])
    # ss = grouped.sum()
    print(f)
    print(ii)
    cmd='tshark -r %s -Y udp -Tfields -e "ip.src" -e "ip.dst" -e "ip.len" -e "udp.srcport"  -e "udp.dstport" -Eseparator=,'%(DATADIR+"/"+f)
    print(cmd)
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE) 
    print("popen")
    d = pd.read_csv(p.stdout, error_bad_lines =False,names=['src','dst','iplen','src_port','dst_port'],dtype=str)
    print("read")
    p.wait()
    print("wait")
    sld=d[(d['src_port']=='123')| (d['dst_port']=='123')]
    sld.to_csv(DATAOUTDIR+"/"+str(ii)+".csv",index=False)
    print("to_csv")

p = mp.Pool(processes=32)

ii=0
for root, dirs, files in os.walk(DATADIR):
    print(root)
    print(dirs)
    print(files)
    for f in files:
        #p.apply_async(worker,(f,ii))
        if f[-7:]=="pcap.gz":
            p.apply_async(worker,(f,ii))
            ii+=1

p.close()
p.join()